In [17]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os

os.environ["LANGSMITH_TRACING"] = "true"


In [79]:
# from langchain_ibm import ChatWatsonx

# model = ChatWatsonx(
#     model_id="ibm/granite-20b-code-instruct",
#     url="https://eu-de.ml.cloud.ibm.com",
#     project_id="aeabba3a-e8d1-4181-bcff-d0a1b639e896"
# )

# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# llm = HuggingFaceEndpoint(
#     repo_id="deepseek-ai/DeepSeek-R1"
# )

# model = ChatHuggingFace(llm=llm)
from langchain_ollama import ChatOllama

model  = ChatOllama(model="qwen3:4b", temperature=0.1)

In [ ]:

from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text:v1.5")


In [73]:
vectors = embeddings.embed_query("How are you")
len(vectors)

768

In [80]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], 
              environment="us-east-1")
              
index_name = "tal-chatbot"
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

Loading Documents


In [75]:
from langchain_community.document_loaders import JSONLoader
file_path = "/Users/sathvika/MCT/SEM-4/industry-project/TAL_Chatbot-1/converters_with_links_and_pricelist.json"

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.[]',  # This will iterate over each item
    text_content=False  # We'll use the entire object as content
)

documents = loader.load()


In [76]:
len(documents)

65

Uploading Embeddings


In [77]:
vectorstore = PineconeVectorStore.from_documents(
    documents,
    embedding=embeddings,
    index_name=index_name
)

Consine Sim Retrieval

In [81]:
from langchain_core.prompts import ChatPromptTemplate

custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a technical assistant for lighting equipment. Always use this JSON context to answer.
     For any query about components:
        1. List ALL matching items from context
        2. Include full technical details
        3. For minimum and maximum questions, refer to the min and max fields respectively
        4. Always respond in markdown, using clear lists and tables where appropriate. Do not output JSON unless specifically asked"""),
        ("human", """Context: {context}
            
        Question: {question}

        Answer:""")
])

In [82]:
from langchain import hub
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"], k=100)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    # messages = prompt.invoke({"question": state["question"], "context": docs_content})
    # response = model.invoke(messages)
    # return {"answer": response.content}
    messages = custom_prompt.invoke({
        "question": state["question"],
        "context": docs_content
    })
    
    response = model.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [83]:
response = graph.invoke({"question": "what lamps can i use for 930606?"})
print(response["answer"])

<think>
Okay, let's see. The user is asking what lamps can be used with the converter model 930606. First, I need to look up the details for that specific converter.

Looking at the data provided, the converter 930606 is described as "POWERLED CONVERTER REMOTE 180mA 9W IP20 CASAMBI" with strain relief and indoor location. The DIMMABILITY is CASAMBI, and the CCR (AMPLITUDE) is YES. The lamps section for this model lists several options.

The lamps listed include Beaufort, Thinksmall halosphere single, Thinksmall halosphere double, Single led XPE, Thinksmall/floorspot WC luxeon MX, *MIX 6 monocolor, Cedrus quantum, *MIX 6 halosphere, MIX 13 monocolor, MIX 13 halosphere, ORBITAL monocolor, ORBITAL halosphere, Beaufort², Haloled, B4, MIX 26 monocolor, and *BOA WC. 

But I need to check the min and max values for each. For example, Beaufort has min 1 and max 1, so that's a single lamp. Thinksmall halosphere single has min 1 to max 2. The *MIX 6 monocolor has min 1 to max 2, and so on. 

How